<a href="https://colab.research.google.com/github/Swifttech-Consult/Machine_Learning101/blob/main/ML_ASSESSMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import os
from google.colab import drive
import tensorflow as tf
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

print("Anthony Akore : anthony.akore@workwithaya.com")

print("ML Assesment Solution")

# Check GPU availability
gpu_info = tf.test.gpu_device_name()
if gpu_info:
    print("GPU:", gpu_info)
else:
    print("No GPU available.")

# Mount Google Drive
drive.mount('/content/gdrive')


# Load dataset from Google Drive with the directory path
dataset_dir = "/content/gdrive/MyDrive/Covid"





# STEP 1
# create, allocate and display images datasets with labels


#1.1
# Create a dataset from the directory
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    image_size=(224, 224),  # Resize to a consistent size
    batch_size=64,
    validation_split=0.8,
    subset="training",
    seed=123
)

#1.2
# Allocate for validation
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    image_size=(224, 224),  # Resize to a consistent size
    batch_size=64,
    validation_split=0.2,
    subset="validation",
    seed=123
)

#1.3
# Iterate through the test dataset and display images with labels
plt.figure(figsize=(20, 20))  # Adjust the figure size
for images, labels in dataset.take(1):  # Take one batch of images for display
    for i in range(min(len(images), 60)):  # View Limited images
        plt.subplot(10, 6, i + 1)  # Create a 10x6grid
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

plt.show()

#1.4
# Define class names (COVID-19 and Normal)
class_names = dataset.class_names

#1.5
# Initialize lists to store true labels and predicted labels
true_labels = []
predicted_labels = []






#STEP 2
# Define , Compile and Train your CNN model


#2.1
# Define your CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification (COVID-19 or not)
])

#2.2
# Define data augmentation in the model
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
])

model = tf.keras.Sequential([
    data_augmentation,
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

#2.3
# Compile your model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#2.4
# Train your model and collect accuracy values
history = model.fit(dataset, epochs=10, validation_data=validation_dataset)

#2.5
# Collect true labels and predicted labels during evaluation
for images, labels in validation_dataset:
    true_labels.extend(labels.numpy())  # Collect true labels
    predictions = model.predict(images)  # Predict labels
    predicted_labels.extend((predictions > 0.5).astype(int).flatten())  # Convert probabilities to binary labels





# STEP 3
# Calculate Accuracy, Store values and Visiualized Graph

#3.1
# Calculate accuracy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#3.2
# Create lists to store accuracy values
train_accuracy = []
val_accuracy = []

#3.3
# Store accuracy values in lists
train_accuracy.extend(history.history['accuracy'])
val_accuracy.extend(history.history['val_accuracy'])

#3.4
# Add Values and Print Accuracy, Precision, Recall and F1 score
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

#3.5
# Plot the accuracy values
plt.figure(figsize=(8, 6))
plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy Over Epochs')
plt.grid(True)
plt.show()

#3.6
# Print true labels and predicted labels for each image
for true_label, predicted_label in zip(true_labels, predicted_labels):
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}")




